# Actinia FUTURES Demo

In [1]:
# Import Python standard library and IPython packages we need.
%load_ext dotenv
%dotenv


import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'
print(gisbase)
ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

/home/coreywhite/.local/grass83


In [6]:
!g.extension extension=ace url=https://github.com/mundialis/ace

/home/coreywhite/.local/grass83/scripts/g.extension:167: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.dir_util import copy_tree
Fetching <ace> from <https://github.com/mundialis/ace/archive/master.zip>
(be patient)...
Compiling...
Installing...
Updating extensions metadata file...
Updating extension modules metadata file...
Installation of <ace> successfully finished


In [65]:
def print_as_json(data):
    print(json.dumps(data, indent=2))

In [300]:
# !g.extension extension=exporter url=https://github.com/mundialis/exporter
!exporter --help

Actinia exporter module supporting raster and vector data.

Usage:
 exporter [raster=name] [vector=name] [--overwrite] [--help]
   [--verbose] [--quiet] [--ui]

Parameters:
  raster   Name of raster map to be exported by actinia
  vector   Name of vector map to be exported by actinia


## Actinia Extension

In [38]:
!ace location="CONUS" create_mapset="futures_test"

Trying to create mapset futures_test
{'accept_datetime': '2022-11-09 05:05:50.139154',
 'accept_timestamp': 1667970350.1391518,
 'api_info': {'endpoint': 'mapsetmanagementresourceadmin',
              'method': 'POST',
              'path': '/api/v3/locations/CONUS/mapsets/futures_test',
              'request_url': 'http://localhost:8088/api/v3/locations/CONUS/mapsets/futures_test'},
 'datetime': '2022-11-09 05:05:50.473532',
 'http_code': 200,
 'message': 'Mapset <futures_test> successfully created.',
 'process_chain_list': [{'1': {'flags': 'l', 'module': 'g.mapsets'}}],
 'process_log': [{'executable': 'g.mapsets',
                  'id': '1',
                  'parameter': ['-l'],
                  'return_code': 0,
                  'run_time': 0.050482749938964844,
                  'stderr': ['Available mapsets:', ''],
                  'stdout': 'PERMANENT dallas_fort_worth_area hydro '
                            'seattle_area washington_dc_area western_nc\n'}],
 'process_resul

In [ ]:
!ace location="the_research_triangle" -m

## Get Mapset info

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/test_mapset/info/" | jq

# Create FUTURES Template

In [293]:
import uuid
PCHAIN = !ace location="CONUS" mapset="futures_test" script="./scripts/FUTURES/load_counties.sh" -d
pchain = json.loads(str("".join(PCHAIN)))
# print_as_json(pchain)
pchain['list'][0]['inputs'][2]['comment'] = "example inpute: geoid in ('48113','48397','48085','48439','48121')"
pchain['list'][0]['inputs'][2]['value'] = "geoid in {{counties}}"
    

observered_pop = {
    "import_descr": {
        "source": "https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/observed_population_counties_NLCD_2001-2019.csv",
        "type": "file"
    },
    "param": "csv",
    "value": "$file::observed_population.csv"
}
    
projected_pop = {
    "import_descr": {
        "source": "https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/Hauer_2020_2100_counties_SSP2_projections_demand.csv",
        "type": "file"
    },
    "param": "csv",
    "value": "$file::projected_population.csv"
}

importer = {
    "id": "importer_1804289383",
    "module": "importer",
    "inputs": [
        observered_pop,
        projected_pop
    ],
    "outputs": [
        {
            "export": {"type": "file", "format": "CSV"},
            "param": "demand",
            "value": "$file::demand",
        },
        {
            "export": {"type": "file", "format": "CSV"},
            "param": "patch_sizes",
            "value": "$file::patches"
        },
        {
            "export": {"type": "file", "format": "CSV"},
            "param": "output",
            "value": "$file::potential"
        }
    ]
}

updated_list = [importer] + pchain['list'] 
pchain['list'] = updated_list


url = "http://localhost:8088/api/v3/locations/CONUS/process_chain_validation_sync"
# print(json.loads(str(pchain)))
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=pchain,
            headers={"content-type": "application/json; charset=utf-8"}
        )
        
templateUUID = str(uuid.uuid4())
template = {
    "id": templateUUID,
    "description": "Run the FUTURES model for selected U.S. counties.",
    "template": pchain
}
pchain['id'] = templateUUID
url =  "http://localhost:8088/api/v3/actinia_templates"
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            data=json.dumps(template),
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'message': 'Successfully created template "b9514dee-253e-47d9-bb5c-c65bc1a035ac".', 'status': 201}


## Check Template was created

In [2]:
url =  "http://localhost:8088/api/v3/actinia_templates"
# # url = "http://localhost:8088/api/v3/locations/the_research_triangle/process_chain_validation_sync"
r = requests.get(
            url, 
            auth=ACTINIA_AUTH,
            headers={"content-type": "application/json; charset=utf-8"}
        )
print(r)
templateIds = r.json()
print(f"Response: {r.json()}")

<Response [200]>
Response: ['f3300b76-9843-46ab-be4d-1c31ae5af97c', 'b9514dee-253e-47d9-bb5c-c65bc1a035ac']


## Request Template

In [3]:
url =  f"http://localhost:8088/api/v3/actinia_templates/{templateIds[1]}"
# # url = "http://localhost:8088/api/v3/locations/the_research_triangle/process_chain_validation_sync"
r = requests.get(
            url, 
            auth=ACTINIA_AUTH,
            headers={"content-type": "application/json; charset=utf-8"}
        )
templateIdResponse = r.json()
# print(f"Response: {r.json()}")
templateIdResponse
templateIdResponse['template']['list'][1]['inputs'][2]['value'] = "('48113')"
templateIdResponse

{'description': 'Run the FUTURES model for selected U.S. counties.',
 'id': 'b9514dee-253e-47d9-bb5c-c65bc1a035ac',
 'template': {'id': 'b9514dee-253e-47d9-bb5c-c65bc1a035ac',
  'list': [{'id': 'importer_1804289383',
    'inputs': [{'import_descr': {'source': 'https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/observed_population_counties_NLCD_2001-2019.csv',
       'type': 'file'},
      'param': 'csv',
      'value': '$file::observed_population.csv'},
     {'import_descr': {'source': 'https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/Hauer_2020_2100_counties_SSP2_projections_demand.csv',
       'type': 'file'},
      'param': 'csv',
      'value': '$file::projected_population.csv'}],
    'module': 'importer',
    'outputs': [{'export': {'format': 'CSV', 'type': 'file'},
      'param': 'demand',
      'value': '$file::demand'},
     {'export': {'format': 'CSV', 'type': 'file'},
      'param': 'patch_sizes',
      'value': '$file::patches'},
     {'expo

## Run Processing Chain

In [ ]:
# load road data
# PCHAIN = !ace location="CONUS" mapset="futures_test" script="./scripts/FUTURES/load_counties.sh" -d
url =  "http://localhost:8088/api/v3/locations/CONUS/mapsets/futures_test/processing_async"
# url =  "http://localhost:8088/api/v3/locations/CONUS/processing_async_export"


r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=pchain,
            headers={"content-type": "application/json; charset=utf-8"}
        )

jsonResponse = r.json()
print(f"Response: {r.json()}")

### Check Processing Chain Status

In [ ]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

## Load OSM PBF data

In [ ]:
osmium tags-filter -v -o us-roads-hospital.osm.pbf us-latest.osm.pbf w/highway r/boundary=administrative nw/amenity=hospital
osm2pgsql -O flex -S streets_hospitals.lua -c -s -d osm -H localhost -U osmuser -P 5431 -W us-roads-hospital.osm.pbf
!osm2pgsql -c -s -d osm -H localhost -U osmuser -P 5431 -W scripts/FUTURES/inputs/north-america-latest.osm.pbf

## 1. Run FUTURES For Testing

In [ ]:
# !ace location="the_research_triangle" mapset="PERMANENT" script="./scripts/FUTURES/load_counties.sh" -d
PCHAIN = !ace location="the_research_triangle" mapset="PERMANENT" script="./scripts/FUTURES/load_counties.sh" -d
url =  "http://localhost:8088/api/v3/locations/the_research_triangle/mapsets/PERMANENT/processing_async"
# url = "http://localhost:8088/api/v3/locations/the_research_triangle/process_chain_validation_sync"
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=json.loads(str("".join(PCHAIN))),
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

In [ ]:
RESPONSE_ID = 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-4a9763f6-9b42-44b8-82a4-8ecc88c70de5' #jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

In [ ]:
!ace -l
!g.extension extension=importer url=https://github.com/mundialis/importer
# !importer --help

# Export CSV test

In [318]:
# !ace location="CONUS" script="./scripts/FUTURES/export_potential_csv.sh" -d
PCHAIN = !ace location="CONUS" script="./scripts/FUTURES/export_potential_csv.sh" -d
pchain = json.loads(str("".join(PCHAIN)))

# Potential
pchain['list'][1]['outputs'][0]['export'] = {"type": "file", "format": "CSV"}

# Patches
pchain['list'][3]['outputs'][0]['export'] = {"type": "file", "format": "CSV"}


url = "http://localhost:8088/api/v3/locations/CONUS/processing_async_export"
# url =  "http://localhost:8088/api/v3/locations/CONUS/mapsets/futures_test/processing_async"

r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=pchain,
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'accept_datetime': '2022-11-10 21:48:56.258750', 'accept_timestamp': 1668116936.2587478, 'api_info': {'endpoint': 'asyncpersistentresource', 'method': 'POST', 'path': '/api/v3/locations/CONUS/mapsets/futures_test/processing_async', 'request_url': 'http://localhost:8088/api/v3/locations/CONUS/mapsets/futures_test/processing_async'}, 'datetime': '2022-11-10 21:48:56.260370', 'http_code': 200, 'message': 'Resource accepted', 'process_chain_list': [], 'process_results': {}, 'queue': 'local', 'resource_id': 'resource_id-2fa4da21-a5b9-4ec0-9250-3f49c006ca71', 'status': 'accepted', 'time_delta': 0.0016300678253173828, 'timestamp': 1668116936.2603688, 'urls': {'resources': [], 'status': 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-2fa4da21-a5b9-4ec0-9250-3f49c006ca71'}, 'user_id': 'actinia-gdi'}


In [322]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6241  100  6241    0     0  17135      0 --:--:-- --:--:-- --:--:-- 17145
{
  "accept_datetime": "2022-11-10 21:48:56.258750",
  "accept_timestamp": 1668116936.2587478,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/CONUS/mapsets/futures_test/processing_async",
    "request_url": "http://localhost:8088/api/v3/locations/CONUS/mapsets/futures_test/processing_async"
  },
  "datetime": "2022-11-10 21:49:16.064259",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "id": "g.region_1804289383",
          "inputs": [
            {
              "param": "vector",
              "value": "counties@futures_test"
            },
            {
              "param": "align",
              

## 1. Run Futures with Development Scenario

In [47]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/data_prep.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'v.to.rast', 'id': 'v.to.rast_1804289383', 'inputs': [{'param': 'input', 'value': 'protected_areas'}, {'param': 'layer', 'value': '1'}, {'param': 'type', 'value': 'point,line,area'}, {'param': 'use', 'value': 'val'}, {'param': 'value', 'value': '1'}, {'param': 'memory', 'value': '300'}], 'outputs': [{'param': 'output', 'value': 'protected_areas'}]}, {'module': 'r.null', 'id': 'r.null_1804289383', 'inputs': [{'param': 'map', 'value': 'protected_areas'}, {'param': 'null', 'value': '0'}]}, {'module': 'r.mapcalc', 'id': 'r.mapcalc_1804289383', 'inputs': [{'param': 'expression', 'value': 'urban_2001 = if(landuse_2001 >= 21 && landuse_2001 <= 24, 1, if(landuse_2001 == 11 || landuse_2001 >= 90 || protected_areas

## 2. Calculate Predictors

In [29]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/predictors.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.slope.aspect', 'id': 'r.slope.aspect_1804289383', 'inputs': [{'param': 'elevation', 'value': 'elevation'}, {'param': 'format', 'value': 'degrees'}, {'param': 'precision', 'value': 'FCELL'}, {'param': 'zscale', 'value': '1.0'}, {'param': 'min_slope', 'value': '0.0'}, {'param': 'nprocs', 'value': '1'}, {'param': 'memory', 'value': '300'}], 'outputs': [{'param': 'slope', 'value': 'slope'}]}, {'module': 'r.mapcalc', 'id': 'r.mapcalc_1804289383', 'inputs': [{'param': 'expression', 'value': 'water = if(landuse_2016 == 11, 1, null())'}, {'param': 'region', 'value': 'current'}]}, {'module': 'r.grow.distance', 'id': 'r.grow.distance_1804289383', 'inputs': [{'param': 'input', 'value': 'water'}, {'param': 'metric

## 3. Development Pressure

In [131]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/development_pressure.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.mapcalc', 'id': 'r.mapcalc_1804289383', 'inputs': [{'param': 'expression', 'value': 'urban_2001_nonull = if(isnull(urban_2001), 0, urban_2001)'}, {'param': 'region', 'value': 'current'}]}, {'module': 'r.mapcalc', 'id': 'r.mapcalc_1804289383', 'inputs': [{'param': 'expression', 'value': 'urban_2016_nonull = if(isnull(urban_2016), 0, urban_2016)'}, {'param': 'region', 'value': 'current'}]}, {'module': 'r.futures.devpressure', 'id': 'r.futures.devpressure_1804289383', 'flags': 'n', 'inputs': [{'param': 'input', 'value': 'urban_2001_nonull'}, {'param': 'method', 'value': 'gravity'}, {'param': 'size', 'value': '30'}, {'param': 'gamma', 'value': '0.5'}, {'param': 'scaling_factor', 'value': '0.1'}, {'param': 

## 4. Development Potential

Now we find best model for predicting urbanization using [r.futures.potential]((https://grass.osgeo.org/grass7/manuals/addons/r.futures.potential.html)) which wraps an R script. We can run R dredge function to find "best" model. We can specify minimum and maximum number of predictors the final model should use.
Because this can be time consuming, for this tutorial we preselected one combination of predictors.


FUTURES site suitability modeled by r.futures.potential and computed by [r.futures.potsurface](https://grass.osgeo.org/grass7/manuals/addons/r.futures.potsurface.html).
We can now visualize the suitability surface using module r.futures.potsurface. It creates initial development potential (suitability) raster from predictors and model coefficients and serves only for evaluating development Potential model. The values of the resulting raster range from 0 to 1.

In [132]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/development_potential.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.futures.potential', 'id': 'r.futures.potential_1804289383', 'inputs': [{'param': 'input', 'value': 'sampling'}, {'param': 'separator', 'value': 'comma'}, {'param': 'columns', 'value': 'devpressure_30_05_01_2001,road_dens_perc,forest_2001_smooth_perc,dist_to_water_km,slope,dist_to_interchanges_km'}, {'param': 'developed_column', 'value': 'urban_change_clip'}, {'param': 'subregions_column', 'value': 'counties'}, {'param': 'min_variables', 'value': '1'}, {'param': 'nprocs', 'value': '1'}], 'outputs': [{'param': 'output', 'value': 'potential.csv'}]}, {'module': 'r.futures.potsurface', 'id': 'r.futures.potsurface_1804289383', 'inputs': [{'param': 'input', 'value': 'potential.csv'}, {'param': 'subregions', '

## 5. Demand submodel (use logarithmic)

We will use r.futures.demand which derives the population vs. development relation. The relation can be linear/logarithmic/logarithmic2/exponential/exponential approach. Look for examples of the different relations in the manual.

linear: y = A + Bx  
logarithmic: y = A + Bln(x)  
logarithmic2: y = A + B * ln(x - C)        (requires SciPy)  
exponential: y = Ae^(BX)  
exp_approach: y = (1 - e^(-A(x - B))) + C        (requires SciPy)  

The format of the input population CSV files is described in the manual. It is important to have synchronized categories of subregions and the column headers of the CSV files (in our case FIPS number). We can simply generate the list of years (for which demand is computed)with this Python code:

In [133]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/demand.sh" -d

{
  "version": "1",
  "list": [
    {
      "module": "g.region",
      "id": "g.region_1804289383",
      "inputs": [
        {
          "param": "vector",
          "value": "counties"
        },
        {
          "param": "align",
          "value": "landuse_2016"
        }
      ]
    },
    {
      "module": "r.mask",
      "id": "r.mask_1804289383",
      "inputs": [
        {
          "param": "raster",
          "value": "roads"
        },
        {
          "param": "maskcats",
          "value": "0"
        },
        {
          "param": "layer",
          "value": "1"
        }
      ]
    },
    {
      "module": "r.futures.demand",
      "id": "r.futures.demand_1804289383",
      "inputs": [
        {
          "param": "development",
          "value": "urban_2001,urban_2006,urban_2011,urban_2016"
        },
        {
          "param": "subregions",
          "value": "counties"
        },
        {
          "param": "observed_population",
          "value": "popu

In [134]:
PCHAIN = '''{
  "version": "1",
  "list": [
    {
      "module": "g.region",
      "id": "g.region_1804289383",
      "inputs": [
        {
          "param": "vector",
          "value": "counties"
        },
        {
          "param": "align",
          "value": "landuse_2016"
        }
      ]
    },
    {
      "module": "r.mask",
      "id": "r.mask_1804289383",
      "inputs": [
        {
          "param": "raster",
          "value": "roads"
        },
        {
          "param": "maskcats",
          "value": "0"
        },
        {
          "param": "layer",
          "value": "1"
        }
      ]
    },
    {
      "module": "r.futures.demand",
      "id": "r.futures.demand_1804289383",
      "inputs": [
        {
          "param": "development",
          "value": "urban_2001,urban_2006,urban_2011,urban_2016"
        },
        {
          "param": "subregions",
          "value": "counties"
        },
        {
          "param": "observed_population",
          "value": "https://raw.githubusercontent.com/ncsu-landscape-dynamics/futures-model-intro-notebook/98e21bb03884939097e800e51dd58237adfec5cc/population_trend.csv"
        },
        {
          "param": "projected_population",
          "value": "https://raw.githubusercontent.com/ncsu-landscape-dynamics/futures-model-intro-notebook/98e21bb03884939097e800e51dd58237adfec5cc/population_projection.csv"
        },
        {
          "param": "simulation_times",
          "value": "2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038"
        },
        {
          "param": "method",
          "value": "logarithmic,logarithmic2,exp_approach"
        },
        {
          "param": "separator",
          "value": "comma"
        }
      ],
      "outputs": [
        {
          "param": "plot",
          "value": "plot_demand.png"
        },
        {
          "param": "demand",
          "value": "demand.csv"
        }
      ]
    }
  ]
}'''
# "value": "PG:host=db port=5432 dbname=actinia user=actinia password=actinia"
url =  "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async"
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=json.loads(str(PCHAIN)),
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'accept_datetime': '2022-08-15 18:41:35.035730', 'accept_timestamp': 1660588895.0357285, 'api_info': {'endpoint': 'asyncpersistentresource', 'method': 'POST', 'path': '/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async', 'request_url': 'http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async'}, 'datetime': '2022-08-15 18:41:35.037443', 'http_code': 200, 'message': 'Resource accepted', 'process_chain_list': [], 'process_results': {}, 'resource_id': 'resource_id-3c2cd531-5ccf-4995-a764-c9d4825f97d5', 'status': 'accepted', 'time_delta': 0.001722097396850586, 'timestamp': 1660588895.0374417, 'urls': {'resources': [], 'status': 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-3c2cd531-5ccf-4995-a764-c9d4825f97d5'}, 'user_id': 'actinia-gdi'}


In [137]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3976  100  3976    0     0  11779      0 --:--:-- --:--:-- --:--:-- 11798
{
  "accept_datetime": "2022-08-15 18:41:35.035730",
  "accept_timestamp": 1660588895.0357285,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async",
    "request_url": "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async"
  },
  "datetime": "2022-08-15 18:41:42.214779",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "id": "g.region_1804289383",
          "inputs": [
            {
              "param": "vector",
              "value": "counties"
            },
            {
              "param": "align",

## 6. Patch calibration (Skip for now)

We derive patches of new development by comparing historical and latest development using [r.futures.calib](https://grass.osgeo.org/grass7/manuals/addons/r.futures.calib.html).
We can keep only patches with minimum size 2 cells (1800 = 2 x 30 x 30 m). We will derive patch libraries for each county separately (flag -s).

In [138]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/patch_calibration.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.mask', 'id': 'r.mask_1804289383', 'inputs': [{'param': 'raster', 'value': 'roads'}, {'param': 'maskcats', 'value': '0'}, {'param': 'layer', 'value': '1'}]}, {'module': 'r.futures.calib', 'id': 'r.futures.calib_1804289383', 'flags': 'sl', 'inputs': [{'param': 'development_start', 'value': 'urban_2001'}, {'param': 'development_end', 'value': 'urban_2016'}, {'param': 'patch_threshold', 'value': '1800'}, {'param': 'nprocs', 'value': '1'}, {'param': 'random_seed', 'value': '1'}, {'param': 'subregions', 'value': 'counties'}, {'param': 'separator', 'value': 'comma'}], 'outputs': [{'param': 'patch_sizes', 'value': 'patches.csv'}]}, {'module': 'r.mask', 'id': 'r.mask_1804289383', 'flags': 'r', 'inputs': [{'para

In [ ]:
!curl -u 'actinia-gdi:actinia-gdi' -X GET http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-ba6655c8-0c25-44c9-a2c0-bfd01ec2645a| jq

## 7. FUTURES simulation

Now we have all the inputs necessary for running [r.futures.pga](https://grass.osgeo.org/grass7/manuals/addons/r.futures.pga.html):

In [100]:
with open('./outputs/calib.csv') as f:
    disc_factor, comp_mean, comp_range = f.readlines()[1].split(',')[:3]

print(f"disc_factor: {disc_factor}, comp_mean: {comp_mean}, comp_range: {comp_range}")

disc_factor: 0.10, comp_mean: 0.00, comp_range: 0.10


In [172]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/futures_simulation.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.futures.pga', 'id': 'r.futures.pga_1804289383', 'inputs': [{'param': 'developed', 'value': 'urban_2016'}, {'param': 'subregions', 'value': 'counties'}, {'param': 'predictors', 'value': 'road_dens_perc,forest_2001_smooth_perc,dist_to_water_km,slope,dist_to_interchanges_km'}, {'param': 'development_pressure', 'value': 'devpressure_30_05_01_2016'}, {'param': 'n_dev_neighbourhood', 'value': '30'}, {'param': 'development_pressure_approach', 'value': 'gravity'}, {'param': 'gamma', 'value': '0.5'}, {'param': 'scaling_factor', 'value': '0.1'}, {'param': 'devpot_params', 'value': 'potential.csv'}, {'param': 'demand', 'value': 'demand.csv'}, {'param': 'separator', 'value': 'comma'}, {'param': 'patch_sizes', 'val

In [155]:
PCHAIN = '''{
  "version": "1",
  "list": [
    {
      "module": "r.futures.pga",
      "id": "r.futures.pga_1804289383",
      "inputs": [
        {
          "param": "developed",
          "value": "urban_2016"
        },
        {
          "param": "subregions",raster
          "value": "counties"
        },
        {
          "param": "predictors",
          "value": "road_dens_perc,forest_2001_smooth_perc,dist_to_water_km,slope,dist_to_interchanges_km"
        },
        {
          "param": "development_pressure",
          "value": "devpressure_30_05_01_2016"
        },
        {
          "param": "n_dev_neighbourhood",
          "value": "30"
        },
        {
          "param": "development_pressure_approach",
          "value": "gravity"
        },
        {
          "param": "gamma",
          "value": "0.5"
        },
        {
          "param": "scaling_factor",
          "value": "0.1"
        },
        {
          "param": "devpot_params",
          "value": "potential.csv"
        },
        {
          "param": "demand",
          "value": "demand.csv"
        },
        {
          "param": "separator",
          "value": "comma"
        },
        {
          "param": "patch_sizes",
          "value": "patches.csv"
        },
        {
          "param": "num_neighbors",
          "value": "4"
        },
        {
          "param": "discount_factor",
          "value": "0.10"
        },
        {
          "param": "seed_search",
          "value": "probability"
        },
        {
          "param": "compactness_mean",
          "value": "0.00"
        },
        {
          "param": "compactness_range",
          "value": "0.100.10,0.00,0.10,0.96,0.29,0.63"
        },
        {
          "param": "incentive_power",
          "value": "1"
        },
        {
          "param": "random_seed",
          "value": "1"
        }
      ],
      "outputs": [
        {
          "param": "output",
          "value": "final"
        },
        {
          "param": "output_series",
          "value": "step"
        }
      ]
    }
  ]
}'''
# "value": "PG:host=db port=5432 dbname=actinia user=actinia password=actinia"
url =  "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async"
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=json.loads(str(PCHAIN)),
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'accept_datetime': '2022-08-15 19:01:56.876588', 'accept_timestamp': 1660590116.8765862, 'api_info': {'endpoint': 'asyncpersistentresource', 'method': 'POST', 'path': '/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async', 'request_url': 'http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async'}, 'datetime': '2022-08-15 19:01:56.877996', 'http_code': 200, 'message': 'Resource accepted', 'process_chain_list': [], 'process_results': {}, 'resource_id': 'resource_id-14b84fb5-8426-4981-a749-fac4ff3f2aca', 'status': 'accepted', 'time_delta': 0.001416921615600586, 'timestamp': 1660590116.8779953, 'urls': {'resources': [], 'status': 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-14b84fb5-8426-4981-a749-fac4ff3f2aca'}, 'user_id': 'actinia-gdi'}


In [163]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3589  100  3589    0     0   9514      0 --:--:-- --:--:-- --:--:--  9519
{
  "accept_datetime": "2022-08-15 19:01:56.876588",
  "accept_timestamp": 1660590116.8765862,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async",
    "request_url": "http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async"
  },
  "datetime": "2022-08-15 19:03:39.846986",
  "http_code": 200,
  "message": "Processing successfully finished",
  "process_chain_list": [
    {
      "list": [
        {
          "id": "r.futures.pga_1804289383",
          "inputs": [
            {
              "param": "developed",
              "value": "urban_2016"
            },
            {
              "param"

## Postprocessing of multiple stochastic runs

We can run multiple stochastic simulations with [r.futures.parallelpga](https://grass.osgeo.org/grass7/manuals/addons/r.futures.parallelpga.html):

In [171]:
!ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/post_processing.sh"
# !ace location="futures_triangle_nc" mapset="futures_test" script="./scripts/FUTURES/futures_simulation.sh"

http://localhost:8088/api/v3/locations/futures_triangle_nc/mapsets/futures_test/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'landuse_2016'}]}, {'module': 'r.futures.parallelpga', 'id': 'r.futures.parallelpga_1804289383', 'inputs': [{'param': 'nprocs', 'value': '10'}, {'param': 'repeat', 'value': '5'}, {'param': 'developed', 'value': 'urban_2016'}, {'param': 'subregions', 'value': 'counties'}, {'param': 'predictors', 'value': 'road_dens_perc,forest_smooth_perc,dist_to_water_km,slope,dist_to_interchanges_km'}, {'param': 'devpot_params', 'value': 'potential.csv'}, {'param': 'development_pressure', 'value': 'devpressure_30_05_01_2016'}, {'param': 'n_dev_neighbourhood', 'value': '30'}, {'param': 'development_pressure_approach', 'value': 'gravity'}, {'param': 'gamma', 'value': '0.5'}, {'param': 'scaling_factor', 'value': '0.1'}, {'param': 'demand', 'value': 'dem

In [30]:
RESPONSE_ID = "http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-e19c450f-96b2-4ebc-83bc-e1214189ce00"

!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1004  100  1004    0     0   2785      0 --:--:-- --:--:-- --:--:--  2781
{
  "accept_datetime": "2022-12-09 14:00:14.978085",
  "accept_timestamp": 1670594414.9780805,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/hudson_river_authority/mapsets/PERMANENT/processing_async",
    "request_url": "http://actinia-core:8088/api/v3/locations/hudson_river_authority/mapsets/PERMANENT/processing_async"
  },
  "datetime": "2022-12-09 18:49:40.478311",
  "http_code": 200,
  "message": "Running executable r.futures.simulation with parameters ['developed=urban_2019', 'subregions=counties', 'p ... dom_seed=1', 'output=final', 'output_series=step'] for 16350.167617559433 seconds",
  "process_chain_list": [],
  "progress": {
    "num_of_steps": 58,
    "step": 58
  },
  "queue"

In [29]:
!ace kill_job="resource_id-fd19d9d3-05f1-4882-98bb-ffb66c46deea"

{'message': 'Internal Server Error'}


In [10]:
!ace list_jobs=running

resource_id-e19c450f-96b2-4ebc-83bc-e1214189ce00 running 2022-12-09 15:43:53.107945


In [2]:
# Unlock Mapset

url =  "http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-47d5c49f-33a1-4f60-95af-c2bdf205408b"
r = requests.get(
            url, 
            auth=ACTINIA_AUTH,
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
jsonResponse


{'accept_datetime': '2023-03-02 19:15:40.720363',
 'accept_timestamp': 1677784540.720362,
 'api_info': {'endpoint': 'asyncpersistentresource',
  'method': 'POST',
  'path': '/api/v3/locations/durham_county/mapsets/PERMANENT/processing_async',
  'request_url': 'http://actinia-core:8088/api/v3/locations/durham_county/mapsets/PERMANENT/processing_async'},
 'datetime': '2023-03-02 19:17:44.310404',
 'exception': {'message': 'AsyncProcessError:  Error while running executable <r.futures.potential>',
  'traceback': ['  File "/usr/lib/python3.9/site-packages/actinia_core/processing/actinia_processing/ephemeral_processing.py", line 1930, in run\n    self._execute()\n',
   '  File "/usr/lib/python3.9/site-packages/actinia_core/processing/actinia_processing/ephemeral/persistent_processing.py", line 758, in _execute\n    self._execute_process_list(process_list)\n',
   '  File "/usr/lib/python3.9/site-packages/actinia_core/processing/actinia_processing/ephemeral/persistent_processing.py", line 678